In [323]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [324]:
#dir_path = "../process/"
#file= "external_test_data"# "train_data" #
fold_N=2
if fold_N!=None:
    dir_path="../process/folds_process/"
file= f"train_fold_{fold_N}" # f"val_fold_{fold_N}" #
print(file)
logs_df = pd.read_csv(f"{dir_path}3_21{file}_heuristic_tagging.csv", encoding='utf-8')
print(set(logs_df["source"]))
print(logs_df.shape)
logs_df.head()

train_fold_2
{'Android', 'Apache', 'BGL', 'Zookeeper', 'Hadoop'}
(2157, 13)


,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:18.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:55.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
2,jk2_init() Found child 6740 in scoreboard slot 7,Informativo,2005-12-04 04:52:05.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Eventos,Recurrente esperado,48,4.0,-1
3,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:15.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
4,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:52:49.000,6,Apache,workerenv init ok httpd conf properties,Depuración,Sistema operativo,Monitoreo,Recurrente esperado,55,4.0,-1


In [325]:
# Función para calcular la prioridad de un evento
def calcular_prioridad(row):
    prioridad = 0

    # Ponderación por Severidad
    if row['Severidad'] == 'Crítico':
        prioridad += 3  # Alta prioridad
    elif row['Severidad'] == 'Advertencia':
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Origen
    if row['Origen'] in ['Sistema operativo', 'Aplicación', ]:
        prioridad += 3  # Alta prioridad para estos orígenes
    elif row['Origen'] in ['Gestión de energía', 'Conectividad']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 

    # Ponderación por Tipo de Evento
    if row['TipoEvento'] in ['Errores', 'Parada de procesos']:
        prioridad += 3  # Alta prioridad
    elif row['TipoEvento'] in  ['Eventos']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 
        
    # Ponderación por Frecuencia
    if row['Frecuencia'] in ['Recurrente anómalo']:
        prioridad += 3  # Alta prioridad
    elif row['Frecuencia'] in ['Único']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Anomaly
    if row['Anomaly'] == -1:
        prioridad += 2  # Alta prioridad
    else:
        prioridad += 1  # Baja prioridad
        
    return prioridad

# Aplicar la función para calcular la prioridad
logs_df['Prioridad'] = logs_df.apply(calcular_prioridad, axis=1)


In [326]:
T=logs_df[logs_df["Level"]=="Error"].copy()
print("Error", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

T=logs_df[logs_df["Level"]=="Advertencia"].copy()
print("Advertencia", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

T=logs_df[logs_df["Level"]=="Informativo"].copy()
print("Informativo", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

Error 5 10.186369958275382 13
Advertencia 5 9.349095966620306 10
Informativo 5 7.119610570236439 10


In [327]:
# Asignar etiquetas de prioridad en función del puntaje
def asignar_prioridad_etiqueta(row):
    if row['Prioridad'] > 10:
        return 'Error'
    elif row['Prioridad'] >= 8:
        return 'Advertencia'
    else:
        return 'Informativo'


logs_df['PrioridadEtiqueta'] = logs_df.apply(asignar_prioridad_etiqueta, axis=1)
logs_df.head(2)

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:18.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:55.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error


In [328]:
t=logs_df[(logs_df["Level"]=="Error") & (logs_df["PrioridadEtiqueta"]=="Advertencia")]
t

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
148,jk2_init() Can't find child 1566 in scoreboard,Error,2005-12-04 17:43:08.000,3,Apache,t child scoreboard,Depuración,Sistema operativo,Monitoreo,Único,46,17.0,1,8,Advertencia
149,mod_jk child init 1 -2,Error,2005-12-04 17:43:12.000,2,Apache,child init,Depuración,Aplicación,Eventos,Recurrente esperado,22,17.0,1,8,Advertencia
189,jk2_init() Can't find child 2082 in scoreboard,Error,2005-12-04 20:47:16.000,3,Apache,t child scoreboard,Depuración,Sistema operativo,Monitoreo,Único,46,20.0,1,8,Advertencia
190,mod_jk child init 1 -2,Error,2005-12-04 20:47:16.000,2,Apache,child init,Depuración,Aplicación,Eventos,Recurrente esperado,22,20.0,1,8,Advertencia
192,jk2_init() Can't find child 2087 in scoreboard,Error,2005-12-04 20:47:17.000,3,Apache,t child scoreboard,Depuración,Sistema operativo,Monitoreo,Único,46,20.0,1,8,Advertencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,program interrupt: privileged instruction...0,Error,NaN,4,BGL,program interrupt privileged instruction,Crítico,Aplicación,Monitoreo,Recurrente esperado,45,NaN,1,9,Advertencia
1941,ciod: LOGIN chdir(/home/germann2/SPaSM_static)...,Error,NaN,7,BGL,ciod login chdir home failed file directory,Advertencia,Sistema de archivos,Errores,Único,80,NaN,1,9,Advertencia
1945,exception syndrome register: 0x00800000,Error,NaN,3,BGL,exception syndrome register,Advertencia,Sistema de archivos,Errores,Recurrente esperado,39,NaN,1,8,Advertencia
1950,ciod: LOGIN chdir(/bglscratch/bwallen/SWL/SYS-...,Error,NaN,15,BGL,ciod login chdir bglscratch bwallen swl sys ca...,Advertencia,Sistema de archivos,Errores,Único,121,NaN,1,9,Advertencia


In [329]:
#t['Clean_Content']

In [330]:
logs_df.to_csv(f"{dir_path}3_22{file}_heuristic_prediction.csv", index=False, encoding='utf-8')

In [331]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [332]:
print({file})
# Asumiendo que logs_df ya está definido y contiene las columnas 'Level' y 'PrioridadEtiqueta'
# Asegúrate de que ambos sean de tipo categórico (opcional pero recomendado)
logs_df['Level'] = logs_df['Level'].astype('category')
logs_df['PrioridadEtiqueta'] = logs_df['PrioridadEtiqueta'].astype('category')

# Extraer columnas
y_true = logs_df['Level']
y_pred = logs_df['PrioridadEtiqueta']

# Calcular accuracy
accuracy = accuracy_score(y_true, y_pred)

# Obtener etiquetas únicas combinadas para garantizar alineación
all_labels = sorted(set(y_true.unique()) | set(y_pred.unique()))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred, labels=all_labels)

# Reporte de clasificación
report = classification_report(y_true, y_pred, labels=all_labels, output_dict=True)

# Convertir a DataFrame
report_df = pd.DataFrame(report).transpose()

# Mostrar resultados
print(f"Accuracy: {accuracy:.2f}\n")
print("Classification Report:")
print(report_df)
print("\nConfusion Matrix:")
print(pd.DataFrame(conf_matrix, index=all_labels, columns=all_labels))

{'train_fold_2'}
Accuracy: 0.72

Classification Report:
              precision    recall  f1-score      support
Advertencia    0.594619  0.922114  0.723010   719.000000
Error          1.000000  0.717663  0.835628   719.000000
Informativo    0.709125  0.518776  0.599197   719.000000
accuracy       0.719518  0.719518  0.719518     0.719518
macro avg      0.767915  0.719518  0.719278  2157.000000
weighted avg   0.767915  0.719518  0.719278  2157.000000

Confusion Matrix:
             Advertencia  Error  Informativo
Advertencia          663      0           56
Error                106    516           97
Informativo          346      0          373


In [333]:
dir_path_model = "../model/"
pd.DataFrame(report).transpose().to_csv(f"{dir_path_model}/rh_report_fold_{fold_N}.csv")
